<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using SingleStore for Iceberg Catalog Storage with PyIceberg</h1>
    </div>
</div>

In [5]:
!pip cache purge --quiet

In [6]:
!pip install "pyiceberg[pandas]"==0.6.1 --quiet

In [7]:
import os
import pandas as pd
import pyarrow as pa
import shutil

from pyiceberg.catalog.sql import SqlCatalog
from pyiceberg.exceptions import NoSuchTableError
from singlestoredb.management import get_secret

In [8]:
os.makedirs("warehouse", exist_ok = True)

In [9]:
url = "https://gist.githubusercontent.com/VeryFatBoy/9af771d443f5ec4dd6eec8d69a062638/raw/c03ef25a97f23a48ee408ac02114195b663a2364/iris.csv"

pandas_df = pd.read_csv(url)

In [10]:
print(pandas_df.head())

   sepal_length  sepal_width  petal_length  petal_width      species
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


In [12]:
%%sql
DROP DATABASE IF EXISTS iris_db;
CREATE DATABASE IF NOT EXISTS iris_db;

1 rows affected.

++
||
++
++

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook.</p>
    </div>
</div>

In [14]:
%%sql
DROP TABLE IF EXISTS iris;

CREATE TABLE IF NOT EXISTS iris (
    sepal_length FLOAT,
    sepal_width FLOAT,
    petal_length FLOAT,
    petal_width FLOAT,
    species VARCHAR(20)
);

++
||
++
++

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [15]:
from sqlalchemy import *

db_connection = create_engine(connection_url)
url = db_connection.url

In [16]:
pandas_df[pandas_df["species"] == "Iris-virginica"].to_sql(
    "iris",
    con = db_connection,
    if_exists = "append",
    index = False
)

50

In [17]:
%%sql
SELECT COUNT(*) FROM iris;

1 rows affected.

COUNT(*)
50


In [18]:
%%sql
SELECT * FROM iris LIMIT 5;

5 rows affected.

sepal_length,sepal_width,petal_length,petal_width,species
7.2,3.0,5.8,1.6,Iris-virginica
5.8,2.7,5.1,1.9,Iris-virginica
6.0,3.0,4.8,1.8,Iris-virginica
6.4,2.8,5.6,2.2,Iris-virginica
6.4,3.2,5.3,2.3,Iris-virginica


In [19]:
password = get_secret("password")

In [20]:
config = {
    "uri": f"singlestoredb://admin:{password}@{url.host}:{url.port}/{url.database}",
    "warehouse": "warehouse",
}

catalog = SqlCatalog(
    name = "s2_catalog",
    **config
)

catalog.create_namespace("default")

In [21]:
table = "iris"
table_identifier = f"default.{table}"

try:
    catalog.drop_table(table_identifier)
    print(f"Dropped table {table_identifier} successfully")
except NoSuchTableError:
    print(f"Table {table_identifier} does not exist. Skipping drop.")
except Exception as e:
    print(f"Error dropping table: {e}")

Table default.iris does not exist. Skipping drop.


In [22]:
namespaces = catalog.list_namespaces()

for namespace in namespaces:
    tables = catalog.list_tables(namespace)
    if tables:
        print(f"Namespace: {namespace}")
        for table in tables:
            print(f"- {table}")
    else:
        print(f"Namespace: {namespace} (empty)")

Namespace: ('default',) (empty)


In [23]:
df = pa.Table.from_pandas(pandas_df)

table = catalog.create_table(
    table_identifier,
    schema = df.schema
)

In [24]:
table.append(df)

len(table.scan().to_arrow())

150

In [25]:
df = table.scan(limit = 5)

print(df.to_pandas())

   sepal_length  sepal_width  petal_length  petal_width      species
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


In [26]:
df = table.scan(row_filter = "species != 'Iris-virginica'").to_arrow()

In [27]:
table.overwrite(df)

len(table.scan().to_arrow())

100

In [28]:
arrow_table = table.scan().to_arrow()

species_counts = arrow_table["species"].value_counts()

print(species_counts.to_pandas())

0        {'values': 'Iris-setosa', 'counts': 50}
1    {'values': 'Iris-versicolor', 'counts': 50}
dtype: object


In [29]:
df = table.scan(row_filter = "species = 'Iris-virginica'")

print(df.to_pandas())

Empty DataFrame
Columns: [sepal_length, sepal_width, petal_length, petal_width, species]
Index: []


In [30]:
new_df = pd.read_sql(
    "SELECT * FROM iris WHERE species = 'Iris-virginica'",
    con = db_connection
)

In [32]:
table.append(pa.Table.from_pandas(new_df))

len(table.scan().to_arrow())

150

In [34]:
arrow_table = table.scan().to_arrow()

species_counts = arrow_table["species"].value_counts()

print(species_counts.to_pandas())

0     {'values': 'Iris-virginica', 'counts': 50}
1        {'values': 'Iris-setosa', 'counts': 50}
2    {'values': 'Iris-versicolor', 'counts': 50}
dtype: object


In [35]:
%%sql
SHOW TABLES;

3 rows affected.

Tables_in_iris_db
iceberg_namespace_properties
iceberg_tables
iris


## Cleanup

In [36]:
shutil.rmtree("warehouse")

In [38]:
%%sql
DROP TABLE IF EXISTS iris;
DROP TABLE IF EXISTS iceberg_tables;
DROP TABLE IF EXISTS iceberg_namespace_properties;

++
||
++
++

In [39]:
%%sql
DROP DATABASE IF EXISTS iris_db;

++
||
++
++